In [33]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
from tsetlin import TsetlinMachine
import torch
import random

DATASET_DIR = '../datasets/'
DATA_FILE = 'bit_1.txt'
SEED = 17648192976567996066

text_rows = open(f'{DATASET_DIR}{DATA_FILE}', 'r').read().splitlines()
dataset = [ [int(num) for num in row.split(',')] for row in text_rows]
tensor_dataset = torch.tensor(dataset)
train_x = tensor_dataset[:, :-1]
train_y = tensor_dataset[:, -1]

if SEED:
    random.seed(SEED)
    torch.manual_seed(SEED)
else:
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    print(seed)
tm = TsetlinMachine(train_x.shape[1], 5)
out_1 = tm.forward(train_x)
out_1, train_y

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


(tensor([0, 1, 0, 0, 0, 0, 0, 0]), tensor([1, 1, 1, 1, 0, 0, 0, 0]))

In [34]:
Y = torch.randint(0,2, tm.l1.out.size())
tm.l1.out, Y, tm.l1.W

(tensor([[0, 0, 0, 0, 0],
         [1, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 1, 1, 0, 1],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0],
         [1, 0, 0, 1, 1],
         [0, 0, 0, 1, 1]]),
 tensor([[0, 0, 0, 0, 0],
         [0, 0, 1, 1, 0],
         [1, 1, 1, 1, 0],
         [1, 0, 1, 1, 1],
         [0, 0, 0, 1, 1],
         [0, 0, 0, 1, 0],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 0, 1]]),
 tensor([[0, 0, 1, 0, 1, 0],
         [1, 1, 1, 0, 0, 0],
         [1, 1, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0, 0]]))

In [35]:
flip_deps = []
for i, single_x in enumerate(tm.l1.full_X):
    flip_dep_row = [[] for _ in range(tm.l1.in_dim * 2)]
    single_Y = Y[i]
    one_Y_idxs = torch.nonzero(single_Y == 1).squeeze(1)
    W_halves = torch.split(tm.l1.W, tm.l1.in_dim, dim=1)
    pos_W = W_halves[0]
    neg_W = W_halves[1]
    for pos_one_Y_idx in one_Y_idxs:
        w_1 = pos_W[pos_one_Y_idx]
        for neg_one_Y_idx in one_Y_idxs:
            w_2 = neg_W[neg_one_Y_idx]
            deps = ((w_1 == w_2) & (w_1 == 1))
            if deps.any():
                dep_idxs = deps.nonzero(as_tuple=True)[0]
                for idx in dep_idxs:
                    if pos_one_Y_idx.item() not in flip_dep_row[idx]:
                        flip_dep_row[idx].append(pos_one_Y_idx.item())
                    if neg_one_Y_idx.item() not in flip_dep_row[idx + tm.l1.in_dim]:
                        flip_dep_row[idx + tm.l1.in_dim].append(neg_one_Y_idx.item())
    flip_deps.append(flip_dep_row)
flip_deps

[[[], [], [], [], [], []],
 [[2], [], [], [3], [], []],
 [[1, 2], [1, 2], [], [3], [0], []],
 [[2], [2], [], [3], [0], []],
 [[], [], [], [], [], []],
 [[], [], [], [], [], []],
 [[1, 2], [1, 2], [], [3], [0], []],
 [[], [1, 2], [], [], [0], []]]

In [36]:
flip_deps_full = []
for i, single_x in enumerate(tm.l1.full_X):
    flip_dep_row = [[] for _ in range(tm.l1.in_dim * 2)]
    single_Y = Y[i]
    one_Y_idxs = torch.nonzero(single_Y == 1).squeeze(1)
    W_halves = torch.split(tm.l1.W, tm.l1.in_dim, dim=1)
    pos_W = W_halves[0]
    neg_W = W_halves[1]
    for pos_one_Y_idx in one_Y_idxs:
        w_1 = pos_W[pos_one_Y_idx]
        for neg_one_Y_idx in one_Y_idxs:
            w_2 = neg_W[neg_one_Y_idx]
            pos_one_W_idxs = torch.nonzero(w_1 == 1).squeeze(1)
            neg_one_W_idxs = torch.nonzero(w_2 == 1).squeeze(1)
            for idx in pos_one_W_idxs:
                if pos_one_Y_idx.item() not in flip_dep_row[idx.item()]:
                    flip_dep_row[idx.item()].append(pos_one_Y_idx.item())
            for idx in neg_one_W_idxs:
                if neg_one_Y_idx.item() not in flip_dep_row[idx.item() + tm.l1.in_dim]:
                    flip_dep_row[idx.item() + tm.l1.in_dim].append(neg_one_Y_idx.item())
            # deps = ((w_1 == w_2) & (w_1 == 1))
            # if deps.any():
            #     dep_idxs = deps.nonzero(as_tuple=True)[0]
            #     for idx in dep_idxs:
            #         if pos_one_Y_idx.item() not in flip_dep_row[idx]:
            #             flip_dep_row[idx].append(pos_one_Y_idx.item())
            #         if neg_one_Y_idx.item() not in flip_dep_row[idx + tm.l1.in_dim]:
            #             flip_dep_row[idx + tm.l1.in_dim].append(neg_one_Y_idx.item())
    flip_deps_full.append(flip_dep_row)
flip_deps_full

[[[], [], [], [], [], []],
 [[2], [2], [2], [3], [], []],
 [[1, 2], [1, 2], [0, 1, 2], [3], [0], []],
 [[2], [2], [0, 2, 4], [3], [0], []],
 [[], [], [4], [3], [], []],
 [[], [], [], [3], [], []],
 [[1, 2], [1, 2], [0, 1, 2, 4], [3], [0], []],
 [[1, 2], [1, 2], [0, 1, 2, 4], [], [0], []]]

In [37]:
tm.l1.full_X

tensor([[1, 0, 0, 0, 1, 1],
        [1, 0, 1, 0, 1, 0],
        [1, 1, 0, 0, 0, 1],
        [1, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 1],
        [0, 1, 0, 1, 0, 1],
        [0, 0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0, 0]])

In [38]:
W_deps = []
for i, single_W in enumerate(tm.l1.W):
    single_Y = Y[:, i]
    single_out = tm.l1.out[:, i]
    one_Y_idxs = torch.nonzero(single_Y == 1).squeeze(1)
    x_prod = tm.l1.full_X[one_Y_idxs].prod(dim=0)    
    must_be_one_idxs = torch.nonzero(x_prod == 1).squeeze(1)
    zero_Y_idxs = torch.nonzero(single_Y == 0).squeeze(1)
    must_be_flipped_row = [[] for _ in range(tm.l1.in_dim * 2)]

    if len(must_be_one_idxs) == 0:
        must_be_one_idxs = torch.arange(tm.l1.in_dim * 2)

        x_stuff = tm.l1.full_X[one_Y_idxs] == 0
        for row in range(x_stuff.shape[0]):
            for col in range(x_stuff.shape[1]):
                if x_stuff[row, col]:
                    must_be_flipped_row[col].append(one_Y_idxs[row].item())


    x_stuff = tm.l1.full_X[zero_Y_idxs][:, must_be_one_idxs] == 1
    for row in range(x_stuff.shape[0]):
        for col in range(x_stuff.shape[1]):
            if x_stuff[row, col] and zero_Y_idxs[row].item() not in must_be_flipped_row[must_be_one_idxs[col]]:
                must_be_flipped_row[must_be_one_idxs[col]].append(zero_Y_idxs[row].item())


    W_deps.append([must_be_one_idxs.tolist(), must_be_flipped_row])

W_deps

[[[0, 1, 2, 3, 4, 5],
  [[6, 7, 0, 1],
   [6, 5],
   [2, 1],
   [2, 3, 4, 5],
   [2, 3, 7, 0, 1, 4],
   [3, 6, 7, 0, 4, 5]]],
 [[0, 1, 2, 3, 4, 5],
  [[6, 7, 0, 1, 3],
   [6, 3, 5],
   [2, 1, 3],
   [2, 4, 5],
   [2, 7, 0, 1, 4],
   [6, 7, 0, 4, 5]]],
 [[0, 1, 2, 3, 4, 5],
  [[6, 7, 0],
   [1, 6, 5],
   [2],
   [1, 2, 3, 4, 5],
   [2, 3, 7, 0, 4],
   [1, 3, 6, 7, 0, 4, 5]]],
 [[0, 1, 2, 3, 4, 5],
  [[4, 5, 6, 0],
   [1, 4, 6, 7],
   [2, 4, 5, 7],
   [1, 2, 3, 7],
   [2, 3, 5, 0],
   [1, 3, 6, 0]]],
 [[0, 1, 2, 3, 4, 5],
  [[4, 6, 7, 0, 1, 2],
   [4, 6, 2, 5],
   [4, 1],
   [3, 5],
   [3, 7, 0, 1],
   [3, 6, 7, 0, 2, 5]]]]